
# ICS - Travaux Pratiques 9

30.04.25


<font color=grey>_Ce 9e TP se concentre sur les différents algorithmes de recherche et de tri que nous avons vus en cours. Nous en profiterons pour aborder d'autres algorithmes similaire, ou de créer nos propres algorithmes sur la base d'instructions. Bien que cette série soit sous forme de Jupyter Notebook, vous devrez résoudre une partie des exercices sur papier._ 

_Note : ne vous sentez pas obligé.e.s de finir la série pendant la séance! Celle-ci est conçue de manière à ce que vous gardiez certains exercices pour vos révisions._ </font>

***

## Exercice 0 : Echauffement

Ordonner (sans justification) les fonctions suivantes par ordre de croissance, en
groupant ensemble les fonctions qui ont le même ordre de croissance. Vous pouvez
vérifier vos réponses en dessinant le graphes de ces fonctions à l'aide du module
`matplotlib` comme au TP précédent.

$$n \log_2 (n), \; (\log_2(n))^2, \; n, \; n^2, \; 2^n, \; 1000n^{10}, \; \sqrt{n}, \;
(\log_2(n^2)), \; \sqrt{n} \log_2 (n), \; \log_2 (n).$$


<font color=grey> S'aider des affirmations suivantes :
- Si $\lim_{n \rightarrow \infty}\frac{g(n)}{f(n)} = +\infty$ alors $f(n) = \mathcal{O}(g(n))$ 
- Si $\lim_{n \rightarrow \infty}\frac{f(n)}{g(n)} = 0$ alors  $f(n) = \mathcal{O}(g(n))$
</font>

***

## Exercice 1 : Recherche de la première occurence d'un nombre

On se propose d'écrire un algorithme qui prend en entrée une liste de nombres `L` **triée** et une valeur `x`, et retourne l'indice de la **première occurrence** de `x` dans la liste. Si aucun élément de valeur `x` n'est dans la liste, l'algorithme doit retourner `None`.

Par exemple, pour la liste `[10, 20, 20, 20, 20, 30, 30]` et la valeur `20` en entrée, l'algorithme doit retourner l'indice `1` (alors que `recherche_binaire` retournerait l'indice `3`).

- Implémentez en Python l'algorithme suivant, qui résoud ce problème:
	- Appeler `recherche_binaire(L, x)`
	- Si cet appel retourne un indice `i`, parcourir `L` en allant à gauche depuis `i` jusqu'à trouver un élément différent de `x` (ou jusqu'à arriver au début de la liste)
	- Retourner l'indice `i` de la première occurrence de `x` dans `L`.
    Quel est l'ordre de croissance du temps de parcours de cet algorithme au pire des cas, en fonction de la taille $n$ de la liste `L`?

- Donnez un autre algorithme qui résoud le même problème mais dont le temps de parcours au pire des cas est de l'ordre de $\log_2(n)$. <font color=grey> Pour cela, aidez-vous de la structure de l'algorithme de recherche binaire. Comment être sûr.e que le nombre trouvé est le premier nombre de la liste?</font>
- Utilisez la librarie `Matplotlib` pour comparer le temps de parcours de vos deux algorithmes sur des listes de taille $n$, pour $n$ de plus en plus grand. Vous pouvez utiliser des listes qui contiennent une même valeur répétée $n$ fois et rechercher la première occurrence de cette valeur (qui devrait être à l'indice `0`).

In [ ]:
# Réponse au point 1.


In [ ]:
# Réponse au point 2.


In [ ]:
# Réponse au point 3.


***

## Exercice 2 : Fibonacci

En cours, nous avons présenté l'implémentation suivante d'un algorithme calculant le $N^e$ nombre de la suite de Fibonacci. 


In [ ]:
def fib_rec(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib_rec(n-1) + fib_rec(n-2)


Cet algorithme ayant un temps de parcours asymptotiquement exponentiel, nous souhaiterions l'améliorer. 

Réécrire l'algorithme précédent afin que sa complexité devienne linéaire $(\Theta(n))$.


     


<font color=grey>Indice : utilisez un algorithme itératif, et non pas récursif ! De quelles éléments de la suite avez-vous besoin pour construire l'élément</font> $f_n$?


In [ ]:
# Votre réponse ici

***

## Exercice 3 : compréhension des algorithmes de tri

- Simulez à la main le parcours de `tri_par_selection` sur la liste ci-dessous en affichant le contenu de la liste à la fin de chaque itération de la boucle `for` extérieure. Vérifiez votre réponse en  appelant `tri_par_selection` sur la liste en entrée dans la cellule ci-dessous (et en insérant une instruction `print` dans le code).

    Essayez d'implémenter l'algorithme sans utiliser vos notes de cours!
	
    `[7, 2, 13, -5, -3, 17, 8, 24, -11, 1]`
	
- Même question pour `tri_par_insertion`.

In [ ]:
# Réponse au point 1.


In [ ]:
# Réponse au point 2.


***

## Exercice 4 : tri à bulles

Le **tri à bulles** (bubble sort) fonctionne de la manière suivante:

Il prend en entrée une liste `L` de taille $n$. Il parcourt la liste $n$ fois. A chaque parcours, dès qu'il rencontre deux éléments adjacents de la liste qui sont dans le mauvais ordre, c'est-à-dire `L[i], L[i+1]` tels que `{L[i] > L[i+1]`, il les échange.

On appelle cet algorithme tri à bulles car à chaque parcours de la liste, le plus grand élément (restant) se déplace vers la fin de la liste comme une bulle qui remonte à la surface de l'eau.

- Observez une simulation du tri à bulles sur l'un des sites proposés en cours, par exemple https://visualgo.net/bn/sorting, pour comprendre exactement comment fonctionne l'algorithme.
- Donnez l'algorithme du tri à bulles en Python. N'oubliez pas de tester votre code sur de petites listes.
- Quel est le temps de parcours du tri à bulles?


In [ ]:
# Réponse au point 2.

***

## Exercice 5 : exercice d'examen

<font color=greblueen> Chaque semaine, je vous propose un exercice sans corrigé détaillé mais en vous indiquant la réponse finale. Ce sera donc à vous de faire l'effort de résoudre ce problème à l'aide des ressources à disposition : vos notes de cours, les forums sur internet ou même ChatGPT du moment que vous faites l'effort de comprendre le code que vous fournit ce Chatbot! 

Pour résoudre cet exercice, je vous conseille de discuter avec vos camarades.</font>

On donne ci-dessous un algorithme de tri, auquel nous avons rajouté l’instruction
`print(f"{i}e element: {L}")`.

In [ ]:
def mafonction (L):
    n = len (L)
    for i in range (n-1,-1,-1):
        j = i
        while j < n-1 and L[j] > L[j+1]:
            L[j], L[j+1] = L[j+1], L[j]
            j += 1
        print (f"{i}e element : {L}")

- Sans exécuter ce code, saurez-vous dire qu'affichent les instructions suivantes?

    `L = [1, 3, 0, -6, 100 , -1]`

    `mafonction (L)`

- Quel algorithme vu en cours fonctionne selon le même principe que celui-ci?

- Quel est le temps de parcours de `mafonction()`? Justifiez brièvement votre réponse et
utilisez la notation de Landau.

- Quelle propriété satisfait l'instance pour laquelle le temps de parcours est minimal?
	Que vaut celui-ci dans le meilleur des cas? Justifiez brièvement votre réponse et utilisez la notation de Landau.